# Task / Motivation

For the last week or so I have been working through MIT's youtube courses for deep learning to get a better understanding of how it worked. I have seen things like RNNs, CNNs, VAEs, and, my personal favorite -- deep reinforcement learning. 

The goal of this assignment is for educational purposes. I used resources from videos, articles, and more to better understand how these things work. I haven't had a lot of exposure to ML/AI and I thought that this would be an interesting project for me to learn more about what is happening under the hood. 

While it may be difficult to re-construct exactly, the lessons that I learned deep diving into this subject help me have a high level understanding of how some of these models actually work, and how the math/statistics drive the solutions. 

# Setup / Environment

After doing some research I quickly learned that OpenAI created an API named "gym" that is specifically used for reinforcement learning tasks, and it has prebuilt game environments already setup. 

https://www.gymlibrary.dev/content/basic_usage/

In [161]:
#!pip install --upgrade gym
import gym

env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)
print(env)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
<TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>


# Action Space
0: LEFT   
1: DOWN   
2: RIGHT   
3: UP

The reward is given a +1 IFF the player reaches the goal, and 0 otherwise. Notice that in our random experiment below, that it is really rare to get to the end randomly. 


In [162]:
import random
episodes = 25
for episode in range(episodes):
    observation, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1, 2, 3])
        observation, reward, terminated, truncated, info = env.step(action)
        score += reward

        if terminated or truncated:
            done = True
    print(f"Episode: {episode}  Score: {score}")
env.close()

Episode: 0  Score: 0.0
Episode: 1  Score: 0.0
Episode: 2  Score: 0.0
Episode: 3  Score: 1.0
Episode: 4  Score: 0.0
Episode: 5  Score: 0.0
Episode: 6  Score: 0.0
Episode: 7  Score: 0.0
Episode: 8  Score: 0.0
Episode: 9  Score: 0.0
Episode: 10  Score: 0.0
Episode: 11  Score: 0.0
Episode: 12  Score: 0.0
Episode: 13  Score: 0.0
Episode: 14  Score: 0.0
Episode: 15  Score: 0.0
Episode: 16  Score: 0.0
Episode: 17  Score: 0.0
Episode: 18  Score: 0.0
Episode: 19  Score: 0.0
Episode: 20  Score: 0.0
Episode: 21  Score: 0.0
Episode: 22  Score: 0.0
Episode: 23  Score: 0.0
Episode: 24  Score: 0.0


# Q Learning

* This material is for learning purposes and has been better understood thanks to resources such as: https://towardsdatascience.com/q-learning-algorithm-from-explanation-to-implementation-cdbeda2ea187 *

The goal of any reinforcement learning task is to maximize the total rewards an agent gets from its environment through a trial and error process. At each step (or state) that the agent is in, it needs to make a decision (an action) of where it can maximize the reward at that step. In our case in the frozen lake example, at each step we are choosing an action step (left, up, right, down) and are rewarded once we can get to the goal. Our model should learn how to interpret what is a good or bad step. 

In order to make the best action at a step *s*, the agent must find the best probability distribution over which action to take at *s*. 

Q Learning is an algorithm that learns about how to find the optimal (or maximum) Q-value at this step. To make this possible, Q learning stores all the values in a table that is constantly updated at each step based on the old q value, and the new learned value. 

*a* is the learning rate.    
*r* is the reward for taking action *a* at state *s*.    
*\gamma* is the discount factor    
*s'* is the next state after taking the next action    
*a'* is the action that maximized the Q value in the next state   

$
\begin{equation}
Q(s,a) \leftarrow (1 - \alpha) \cdot Q(s,a) + \alpha \cdot \left( r + \gamma \cdot \max_{a'} Q(s',a') \right)
\end{equation}
$

In [163]:
import numpy as np 

qtable = np.zeros((env.observation_space.n, env.action_space.n))
qtable

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Exploration or Exploitation 

https://gist.github.com/aamrani-dev/fe00597615dae967f5ca1909a6ecf1d2#file-q_learning-py

As I learn more about reinforcement learning and Q-learning, it becomes clear that knowing when to explore or exploit is essential. In the early iterations, our reinforcement learning model lacks information about the environment, prompting it to prioritize exploration over exploitation. However, as the model learns with each iteration, the exploration decay constant gradually diminishes the need for extensive exploration! Instead, the model begins to leverage its accumulated knowledge, shifting towards prioritizing exploitation. 

To choose whether to explore or exploit, we use a uniform distribution between 0 and 1 and if our random number is less than the exploration probability, the agent selects a random action (explores), otherwise, exploits the newfound knowledge using the "bellman equation" as explained from the article. (Very similar to how Markov chain Monte Carlo scenarios)

In [171]:
n_episodes = 15000                 
max_iter_episode = 100     
min_exploration_proba = 0.01              
exploration_proba = 1.0                
exploration_decreasing_decay = 0.005
gamma = 0.95                         
lr = 0.8                         

rewards = []
for e in range(n_episodes):
    current_state = env.reset()[0]
    done = False
    
    total_episode_reward = 0
    
    for i in range(max_iter_episode): 
        if np.random.uniform(0,1) < exploration_proba:
            action = env.action_space.sample()  # Exploration
        else:
            action = np.argmax(qtable[current_state,:]) # Exploitation
        
        next_state, reward, terminated, truncated, info = env.step(env.action_space.sample())

        if (terminated or truncated):
            done = True
        
        qtable[current_state, action] = (1-lr) * qtable[current_state, action] + lr * (reward + gamma * max(qtable[next_state, :]))
        total_episode_reward = total_episode_reward + reward
        if done:
            break
        current_state = next_state


    exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay*e))
    rewards.append(total_episode_reward)


In [172]:
print("Mean reward per thousand episodes")
for i in range(10):
    print((i+1)*1000,": mean espiode reward: ", np.mean(rewards[1000*i:1000*(i+1)]))

Mean reward per thousand episodes
1000 : mean espiode reward:  0.017
2000 : mean espiode reward:  0.02
3000 : mean espiode reward:  0.016
4000 : mean espiode reward:  0.009
5000 : mean espiode reward:  0.013
6000 : mean espiode reward:  0.013
7000 : mean espiode reward:  0.008
8000 : mean espiode reward:  0.014
9000 : mean espiode reward:  0.015
10000 : mean espiode reward:  0.029


In [173]:
print(qtable)

[[3.01597476e-05 1.21337457e-05 7.04186646e-06 1.53302129e-05]
 [5.24654053e-07 5.12249747e-07 4.48738417e-07 5.51176790e-07]
 [1.29826986e-06 1.90563507e-06 2.17000462e-06 5.25098727e-06]
 [5.08897780e-06 4.69426206e-07 7.91646348e-07 1.27497473e-06]
 [1.83514974e-07 6.87412232e-06 7.11414411e-04 1.51401163e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.78308478e-09 2.67227212e-04 7.81131272e-11 2.82086320e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.81271653e-05 8.68620664e-06 2.49666371e-03 6.38562702e-06]
 [6.95822017e-05 1.06350360e-05 2.68215530e-05 7.02920352e-04]
 [2.65221281e-05 5.50744598e-05 1.27626441e-05 7.03222006e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.31292037e-05 1.82878903e-02 1.28478193e-02 1.33970200e-03]
 [1.86828242e-02 1.33386564e-02 3.66536333e-01 7.55032010e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000